# Decision Tree Classifier which let's you predict if tweet is positive or negative.


In [1]:
import pandas as pd
import numpy as np
import re
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import requests
import nltk
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import tree
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt

In [ ]:
brexit_df = pd.read_csv("/content/brexitKlasa.csv", encoding='utf-8')
brexit_df = brexit_df.drop_duplicates(subset=['text'])
brexit = brexit_df['text'].tolist()

def clean_text(text):
    text = text.lower()
    text = re.sub("rt"," ", text)
    text = re.sub("#[A-Za-z0-9_]+", " ", text)
    text = re.sub("@[A-Za-z0-9_]+", " ", text)
    text = re.sub('(http|https):\/\/[a-z\.]*\/[a-zA-Z0-9]*', '', text)
    text = re.sub("n/"," ", text)
    text = re.sub("[?!;(),.:'+-=]", " ", text)
    text = re.sub("[0-9]+"," ", text)
    text = re.sub("'"," ", text)
    text = re.sub("'","",text)
    text = re.sub("&"," ", text)
    text = re.sub("…"," ", text)
    text = re.sub("-"," ", text)
    text = re.sub("%"," ", text)
    text = re.sub('"'," ", text)
    text = re.sub('@'," ", text)
    text = re.sub('�'," ", text)
    text = re.sub("[ ]+"," ", text)
    return text.strip()

url = "https://www.textfixer.com/tutorials/common-english-words.txt"
response = requests.get(url)
stopwords = response.text.split(",")
stopwords = stopwords + ['htt', 's', 'c', 'y', 'j', 't', "it’s", 'https', '', 'h', 'w', 'v', 'u', 'k', "won't"]

In [ ]:
clean_brexit = [clean_text(x) for x in brexit]
brexit_sw = []
for i in clean_brexit:
  nowy_tweet = []
  for j in i.split(" "):
    if j not in stopwords2:
      nowy_tweet.append(j)
      n = " ".join(nowy_tweet)
  brexit_sw.append(n)

stemmer = SnowballStemmer("english")
stem_brexit = []
for i in brexit_sw:
  temp = []
  for j in i.split(" "):
    temp.append(stemmer.stem(j))
  stem_brexit.append(" ".join(temp))

In [ ]:
v = TfidfVectorizer(stop_words='english')
x=v.fit_transform(stem_brexit)
df = pd.DataFrame(x.toarray(), columns=v.get_feature_names())

brexit_df = brexit_df.reset_index()
brexit_df = brexit_df.drop(columns=['index'])
df1 = pd.merge(df, brexit_df, left_index=True, right_index=True, how='left')
df1 = df1.drop(columns=['screenName', 'text'])

X = df1.iloc[:, :-1].values
Y = df1.iloc[:, -1].values
min_max_scaler = MinMaxScaler()
X_minmax = min_max_scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_minmax, df1['klasa'].values , test_size=0.30, random_state=0)
model_tree = DecisionTreeClassifier(criterion="entropy", random_state=0)
model_tree.fit(X_train, y_train)
y_pred = model_tree.predict(X_test)

In [ ]:
plt.figure(figsize = [100, 100])
tree.plot_tree(model_tree, fontsize=60, feature_names = v.get_feature_names(), max_depth= 5)
plt.show()

# Classifier accuracy score. Which class is predicted better? 

In [ ]:
print("Calssification accuracy is: ", accuracy_score(y_test, y_pred))

In [ ]:
def whichClass():
  mat = confusion_matrix(y_test, y_pred)
  c = mat.diagonal()/mat.sum(axis=1)
  n = c[0]
  p = c[1]
  if p > n:
    print('Higher accuracy for "pozytywna" class.')
  elif p < n:
    print('Higher accuracy for "negatywna" class.')
  else:
    print('None of two classes has higher accuracy.')

whichClass()

# 3 most discriminatory terms.

In [ ]:
from six import text_type
text_representation = tree.export_text(model_tree, feature_names=v.get_feature_names())
print(text_representation)

In [ ]:
print("3 most discriminatory terms are: 'thing', 'ireland' oraz 'avoid' (based on rules analysis)")